Goal: pull out 1 buoy record that has ssc&chla and search for matching icesat2 granules

To do:
- Load info for 1 relevant station
- Find all matching atl granules and save times from file names
- Compare times to see if any matching buoy times

In [6]:
%pip install --quiet "sliderule @ git+https://github.com/SlideRuleEarth/sliderule#subdirectory=clients/python"

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install --quiet erddapy

Note: you may need to restart the kernel to use updated packages.


In [1]:
from erddapy import ERDDAP
import pandas as pd
import numpy as np
from sliderule import sliderule, icesat2, earthdata
from datetime import datetime, timezone, timedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

sliderule.init(verbose=False)

True

In [2]:
def fname2datetime(fname):
    y = int(fname[6:10])
    m = int(fname[10:12])
    d = int(fname[12:14])
    H = int(fname[14:16])
    M = int(fname[16:18])
    S = int(fname[18:20])

    t = datetime(y,m,d,H,M,S, tzinfo=timezone.utc)
    return t

def buoy_bound_box(lat,lon,buffer_km):
    # define a buffer distance around the buoy to search for icesat-2 data
    lat_buff = buffer_km/111 # convert buffer distance to frac of 1 deg lat
    lon_buff = buffer_km/(111*np.cos(lat*np.pi/180)) # convert buffer distance to frac of 1 deg lon
    # define bounding box around the buoy (WSEN)
    # example: bbox = [-108.3, 39.2, -107.8, 38.8]
    # bbox = [lon-lon_buff,lat+lat_buff,lon+lon_buff,lat-lat_buff]
    # region = sliderule.toregion(bbox)
    minx = lon - lon_buff
    miny = lat - lat_buff
    maxx = lon + lon_buff
    maxy = lat + lat_buff

    poly = [{'lon': minx, 'lat': miny},
            {'lon': maxx, 'lat': miny},
            {'lon': maxx, 'lat': maxy},
            {'lon': minx, 'lat': maxy},
            {'lon': minx, 'lat': miny}] # Closing the loop by repeating the first point
    return poly


In [3]:
# load a list of all the relevant ERDDAPs and their urls
FF = pd.read_pickle("labeled_relevant_stations.pkl")
FF = FF[(FF.buoy == True) & (FF.radiation==True)]
FF.reset_index(drop=True, inplace=True)

FF["geospatial_lat_min"] = pd.to_numeric(FF["geospatial_lat_min"])
FF["geospatial_lon_min"] = pd.to_numeric(FF["geospatial_lon_min"])
FF["geospatial_lat_max"] = pd.to_numeric(FF["geospatial_lat_max"])
FF["geospatial_lon_max"] = pd.to_numeric(FF["geospatial_lon_max"])
FF["photon_data"] = False


In [4]:
# remove stations with PAR data in air or just no good data
FF = FF[19:]
FF = FF.drop(index=[34,35,36,37,39,40])
FF.reset_index(drop=True, inplace=True)
FF

,sites,url,time_coverage_start,time_coverage_end,geospatial_lat_min,geospatial_lat_max,geospatial_lon_min,geospatial_lon_max,geospatial_lat_units,geospatial_lon_units,check_sum,buoy,glider,ship,turb,ssc,phot,chl,turbid,photo,radiation,photon_data
0,ooi-ce01issp-sp001-10-paradj000,https://erddap.dataexplorer.oceanobservatories...,2014-04-17T22:28:00Z,2023-09-22T08:00:00Z,44.660120,44.660120,-124.098120,-124.098120,degrees_north,degrees_east,1.0,True,False,False,False,False,True,False,False,True,True,False
1,ooi-ce04osps-sf01b-3c-parada102,https://erddap.dataexplorer.oceanobservatories...,2015-08-03T15:10:00Z,2024-04-09T22:00:00Z,44.369353,44.369353,-124.954108,-124.954108,degrees_north,degrees_east,1.0,True,False,False,False,False,True,False,False,True,True,False
2,ooi-ce02shsp-sp001-09-paradj000,https://erddap.dataexplorer.oceanobservatories...,2015-03-18T19:36:00Z,2023-09-23T08:08:00Z,44.635550,44.635550,-124.301470,-124.301470,degrees_north,degrees_east,1.0,True,False,False,False,False,True,False,False,True,True,False
3,ooi-ce06issp-sp001-10-paradj000,https://erddap.dataexplorer.oceanobservatories...,2015-04-10T18:28:00Z,2023-09-23T08:02:00Z,47.135530,47.135530,-124.270170,-124.270170,degrees_north,degrees_east,1.0,True,False,False,False,False,True,False,False,True,True,False
4,ooi-ce09ospm-wfp01-05-paradk000,https://erddap.dataexplorer.oceanobservatories...,2014-04-19T01:02:00Z,2024-04-19T13:09:00Z,46.851630,46.851630,-124.974420,-124.974420,degrees_north,degrees_east,1.0,True,False,False,False,False,True,False,False,True,True,False
5,ooi-ce07shsp-sp001-09-paradj000,https://erddap.dataexplorer.oceanobservatories...,2015-04-09T23:25:00Z,2022-04-26T20:04:00Z,46.983580,46.983580,-124.552130,-124.552130,degrees_north,degrees_east,1.0,True,False,False,False,False,True,False,False,True,True,False
6,ooi-cp02pmci-wfp01-05-paradk000,https://erddap.dataexplorer.oceanobservatories...,2014-04-17T01:02:00Z,2022-11-18T18:36:00Z,40.226633,40.226633,-70.889067,-70.889067,degrees_north,degrees_east,1.0,True,False,False,False,False,True,False,False,True,True,False
7,ooi-cp02pmco-wfp01-05-paradk000,https://erddap.dataexplorer.oceanobservatories...,2014-04-13T18:02:00Z,2022-11-18T15:30:00Z,40.096817,40.096817,-70.879150,-70.879150,degrees_north,degrees_east,1.0,True,False,False,False,False,True,False,False,True,True,False
8,ooi-cp01cnpm-wfp01-05-paradk000,https://erddap.dataexplorer.oceanobservatories...,2017-11-06T19:02:00Z,2022-04-24T18:27:00Z,40.133907,40.133907,-70.770433,-70.770433,degrees_north,degrees_east,1.0,True,False,False,False,False,True,False,False,True,True,False
9,ooi-cp03ispm-wfp01-05-paradk000,https://erddap.dataexplorer.oceanobservatories...,2017-11-12T05:02:00Z,2022-04-25T18:24:00Z,40.362005,40.362005,-70.878502,-70.878502,degrees_north,degrees_east,1.0,True,False,False,False,False,True,False,False,True,True,False


In [6]:
# search parameters
search_hrs = 3
search_km = 3
# loop through each buoy asset:
for jj in range(len(FF)):
# for jj in range(4,7):
    print('working on ' + str(jj+1)+'/'+str(len(FF)))
    # define a search region around the buoy 
    lat = FF['geospatial_lat_min'][jj]
    lon = FF['geospatial_lon_min'][jj]
    poly = buoy_bound_box(lat,lon,search_km)

    # search CMR for ATL03 granules in the bounding box
    grns = earthdata.cmr(short_name="ATL03",
                         polygon=poly,
                         version='006')
    # save the times for each granule as a datetime object
    icesat_times = [fname2datetime(fname) for fname in grns]

    # set up erddap search for this server:
    e = ERDDAP(
    server=FF['sites'][jj], 
    protocol="tabledap", # Want table data (not a grid map of data) 
    response="csv") #in csv format for pandas

    # now check if buoy data exists for these granules\n",
    e = ERDDAP(server=FF['url'][jj],
               protocol="tabledap",
               response="csv")
    e.dataset_id = FF['sites'][jj]

    for t in icesat_times:
        # add a time buffer (+/-1 hours) to search for relevant buoy data for each granule,
        t_start = (t-timedelta(hours=search_hrs)).strftime("%Y-%m-%dT%H:%M:%SZ")
        t_end = (t+timedelta(hours=search_hrs)).strftime("%Y-%m-%dT%H:%M:%SZ")
        e.constraints = {"time>=": t_start,
                        "time<=": t_end}

        # try to download the associated buoy data
        try:
            buoy = e.to_pandas()           
        except:
            continue
        # if buoy data exists, download the ATL03 photons in the bounding box at this time
        
        # Build ATL03 Request
        poly = buoy_bound_box(lat,lon,search_km)       
        parms = {"poly": poly,
                 "t0": t_start,
                 "t1": t_end,
                 "track": 0,
                 "len": 20.0,
                 "pass_invalid": True,
                 "cnf": -2, # returns all photons
                 "srt": icesat2.SRT_OCEAN}
        atl_gdb = icesat2.atl03sp(parms)
        
        if len(atl_gdb)>0:            
            FF.loc[jj,"photon_data"] = True            
            #atl_gdb.to_pickle('icesat2_'+str(e.dataset_id)+'_'+str(t_start)+'.pkl')
            print('success '+ e.dataset_id + ' ' + str(t_start))
            print('no. of photons: '+str(len(atl_gdb)))
            buoy.to_csv('data_'+str(e.dataset_id)+'_'+str(t_start)+'.csv')


working on 16/17
working on 17/17
success Gulf-of-Mexico-C12-OCSv3-Data-10252018-to-01292020 2018-11-02T05:10:53Z
no. of photons: 15037
success Gulf-of-Mexico-C12-OCSv3-Data-10252018-to-01292020 2019-03-19T10:37:39Z
no. of photons: 742
success Gulf-of-Mexico-C12-OCSv3-Data-10252018-to-01292020 2019-05-02T20:30:26Z
no. of photons: 255
success Gulf-of-Mexico-C12-OCSv3-Data-10252018-to-01292020 2019-09-17T01:57:05Z
no. of photons: 2376
success Gulf-of-Mexico-C12-OCSv3-Data-10252018-to-01292020 2019-12-16T21:36:54Z
no. of photons: 1669
